# 📘 Agentic Architectures: Episodic + Semantic Memory Stack (Local Setup)

Welcome to this notebook where we implement a structured memory architecture using **100% local infrastructure**:

- **LLM**: Ollama (Docker container)
- **Embeddings**: Ollama Embeddings (Docker container)
- **Graph Database**: Neo4j (Docker container)
- **Vector Store**: Qdrant (Docker container)

No cloud APIs or API keys required!

We will implement a memory architecture that mirrors human cognition, combining two distinct types of memory:

1.  **Episodic Memory:** This is the memory of specific events or past interactions. It answers the question, "What happened?" (e.g., "Last week, the user asked me about NVIDIA's stock price."). We'll use a **vector database** for this to find past conversations relevant to the current topic.
2.  **Semantic Memory:** This is the memory of structured facts, concepts, and relationships extracted from those events. It answers the question, "What do I know?" (e.g., "User Alex is a conservative investor.", "Alex is interested in Tech Stocks."). We'll use a **graph database (Neo4j)** for this, as it excels at managing and querying complex relationships.

By combining these, our agent can not only recall past conversations but also build a rich, interconnected knowledge base about the user and the world, leading to deeply personalized and context-aware interactions.

## Phase 0: Foundation & Setup

We'll install all necessary libraries, including drivers for our vector and graph databases.

In [ ]:
# Uncomment and run if packages are not installed (using uv)
# !uv pip install langchain-ollama langchain langgraph neo4j rich python-dotenv langchain_community

In [13]:
import os
import uuid
from typing import List, Dict, Any, Optional, Tuple
from dotenv import load_dotenv

# Pydantic for data modeling
from pydantic import BaseModel, Field

# LangChain components - Using Ollama for local LLM and embeddings
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_neo4j import Neo4jGraph

from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate

# LangGraph components
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown

# --- Environment Setup ---
load_dotenv()

# Check for required environment variables (no API keys needed for local!)
required_vars = ["NEO4J_URI", "NEO4J_USERNAME", "NEO4J_PASSWORD"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

# Optional: Set up LangSmith tracing if you have it configured
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Memory Stack (Local)"

print("Environment variables loaded.")

Environment variables loaded.


## Phase 1: Building the Memory Components

This is the core of our architecture. We'll define the structures for our memories and set up the connections to our databases. We'll also create the "Memory Maker" agent responsible for processing conversations and creating new memories.

### Ollama Model Configuration

You can change the models below based on what you have pulled:

**LLM Options:** `llama3.2`, `llama3.2:1b`, `mistral`, `qwen2.5`, `gemma2`, `phi3`

**Embedding Options:** `nomic-embed-text`, `mxbai-embed-large`, `all-minilm`

In [14]:
console = Console()

# --- Configuration: Change these based on your Ollama models ---
LLM_MODEL = "qwen3:1.7b"           # or "mistral", "qwen2.5", etc.
EMBEDDING_MODEL = "nomic-embed-text"  # or "mxbai-embed-large", "all-minilm"

# Initialize Ollama LLM and Embeddings (Local Docker container - No API key needed!)
llm = ChatOllama(
    model=LLM_MODEL,
    temperature=0,
    base_url=os.environ.get("OLLAMA_HOST", "http://localhost:11434")
)
embeddings = OllamaEmbeddings(
    model=EMBEDDING_MODEL,
    base_url=os.environ.get("OLLAMA_HOST", "http://localhost:11434")
)

print(f"Using LLM: {LLM_MODEL}")
print(f"Using Embeddings: {EMBEDDING_MODEL}")

# --- 1. Vector Store for Episodic Memory (Qdrant) ---
COLLECTION_NAME = "episodic_memory"
try:
    qdrant_client = QdrantClient(
        host=os.environ.get("QDRANT_HOST", "localhost"),
        port=int(os.environ.get("QDRANT_PORT", 6333))
    )
    
    # Create collection if it doesn't exist
    collections = [c.name for c in qdrant_client.get_collections().collections]
    if COLLECTION_NAME not in collections:
        qdrant_client.create_collection(
            collection_name=COLLECTION_NAME,
            vectors_config=VectorParams(size=768, distance=Distance.COSINE)  # nomic-embed-text dimension
        )
    
    episodic_vector_store = QdrantVectorStore(
        client=qdrant_client,
        collection_name=COLLECTION_NAME,
        embedding=embeddings
    )
    print("✅ Episodic memory (Qdrant) initialized.")
except Exception as e:
    console.print(f"[bold red]Failed to initialize Qdrant: {e}[/bold red]")
    episodic_vector_store = None

# --- 2. Graph DB for Semantic Memory ---
try:
    graph = Neo4jGraph(
        url=os.environ.get("NEO4J_URI"),
        username=os.environ.get("NEO4J_USERNAME"),
        password=os.environ.get("NEO4J_PASSWORD")
    )
    # Clear the graph for a clean run
    graph.query("MATCH (n) DETACH DELETE n")
    print("✅ Semantic memory (Neo4j) connected and cleared.")
except Exception as e:
    console.print(f"[bold red]Failed to connect to Neo4j: {e}. Please check your credentials and that Neo4j is running.[/bold red]")
    graph = None

# --- 3. Pydantic Models for the "Memory Maker" ---
# Define the structure of knowledge we want to extract.
class Node(BaseModel):
    id: str = Field(description="Unique identifier for the node, which can be a person's name, a company ticker, or a concept.")
    type: str = Field(description="The type of the node (e.g., 'User', 'Company', 'InvestmentPhilosophy').")
    properties: Dict[str, Any] = Field(default_factory=dict, description="A dictionary of properties for the node.")

class Relationship(BaseModel):
    source: Node = Field(description="The source node of the relationship.")
    target: Node = Field(description="The target node of the relationship.")
    type: str = Field(description="The type of the relationship (e.g., 'IS_A', 'INTERESTED_IN').")
    properties: Dict[str, Any] = Field(default_factory=dict, description="A dictionary of properties for the relationship.")

class KnowledgeGraph(BaseModel):
    """Represents the structured knowledge extracted from a conversation."""
    relationships: List[Relationship] = Field(default_factory=list, description="A list of relationships to be added to the knowledge graph.")

print("✅ Pydantic models defined.")

Using LLM: qwen3:1.7b
Using Embeddings: nomic-embed-text
✅ Episodic memory (Qdrant) initialized.
✅ Semantic memory (Neo4j) connected and cleared.
✅ Pydantic models defined.


In [15]:
# --- 4. The "Memory Maker" Agent ---
def create_memories(user_input: str, assistant_output: str):
    """Creates both episodic and semantic memories from an interaction."""
    conversation = f"User: {user_input}\nAssistant: {assistant_output}"
    
    # 4a. Create Episodic Memory (Summarization)
    console.print("--- Creating Episodic Memory (Summary) ---")
    summary_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a summarization expert. Create a concise, one-sentence summary of the following user-assistant interaction. This summary will be used as a memory for future recall."),
        ("human", "Interaction:\n{interaction}")
    ])
    summarizer = summary_prompt | llm
    episodic_summary = summarizer.invoke({"interaction": conversation}).content
    
    new_doc = Document(page_content=episodic_summary, metadata={"created_at": uuid.uuid4().hex})
    episodic_vector_store.add_documents([new_doc])
    console.print(f"[green]Episodic memory created:[/green] '{episodic_summary}'")
    
    # 4b. Create Semantic Memory (Fact Extraction)
    console.print("--- Creating Semantic Memory (Graph) ---")
    
    # For local models, we use a more explicit extraction approach
    extraction_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a knowledge extraction expert. Extract key entities and their relationships from the conversation.
        
Return your response as a JSON object with this exact structure:
{{
    "relationships": [
        {{
            "source": {{"id": "entity_name", "type": "EntityType", "properties": {{}}}},
            "target": {{"id": "entity_name", "type": "EntityType", "properties": {{}}}},
            "type": "RELATIONSHIP_TYPE",
            "properties": {{}}
        }}
    ]
}}

Focus on user preferences, goals, stated facts, and interests. Use uppercase for relationship types.
Common entity types: User, Company, Sector, InvestmentPhilosophy, Topic, Preference
Common relationship types: INTERESTED_IN, HAS_GOAL, PREFERS, MENTIONED, ASKED_ABOUT"""),
        ("human", "Extract all relationships from this interaction:\n{interaction}")
    ])
    
    try:
        # Try structured output first (works with some Ollama models)
        try:
            extraction_llm = llm.with_structured_output(KnowledgeGraph)
            extractor = extraction_prompt | extraction_llm
            kg_data = extractor.invoke({"interaction": conversation})
        except Exception:
            # Fallback: Parse JSON from raw response
            import json
            extractor = extraction_prompt | llm
            response = extractor.invoke({"interaction": conversation}).content
            # Try to extract JSON from the response
            json_start = response.find('{')
            json_end = response.rfind('}') + 1
            if json_start != -1 and json_end > json_start:
                json_str = response[json_start:json_end]
                kg_dict = json.loads(json_str)
                kg_data = KnowledgeGraph(**kg_dict)
            else:
                kg_data = KnowledgeGraph(relationships=[])
        
        if kg_data.relationships:
            for rel in kg_data.relationships:
                # Create nodes and relationship in Neo4j
                cypher = """
                MERGE (s:{source_type} {{id: $source_id}})
                MERGE (t:{target_type} {{id: $target_id}})
                MERGE (s)-[r:{rel_type}]->(t)
                """.format(
                    source_type=rel.source.type.replace(' ', '_'),
                    target_type=rel.target.type.replace(' ', '_'),
                    rel_type=rel.type.replace(' ', '_')
                )
                graph.query(cypher, {
                    'source_id': rel.source.id,
                    'target_id': rel.target.id
                })
            console.print(f"[green]Semantic memory created:[/green] Added {len(kg_data.relationships)} relationships to the graph.")
        else:
            console.print("[yellow]No new semantic memories identified in this interaction.[/yellow]")
    except Exception as e:
        console.print(f"[red]Could not extract or save semantic memory: {e}[/red]")

if episodic_vector_store and graph:
    print("\n✅ Memory components initialized successfully!")
else:
    print("\n⚠️ Some memory components failed to initialize. Check errors above.")


✅ Memory components initialized successfully!


## Phase 2: The Memory-Augmented Agent

Now we'll build the agent that uses this memory system. We'll use LangGraph to define a clear, stateful workflow: retrieve memories, generate a response using those memories, and finally, update the memory with the latest interaction.

In [16]:
# Define the state for our LangGraph agent
class AgentState(TypedDict):
    user_input: str
    retrieved_memories: Optional[str]
    generation: str

# Define the nodes of the graph

def retrieve_memory(state: AgentState) -> Dict[str, Any]:
    """Node that retrieves memories from both episodic and semantic stores."""
    console.print("--- Retrieving Memories ---")
    user_input = state['user_input']
    
    # Retrieve from episodic memory
    retrieved_docs = episodic_vector_store.similarity_search(user_input, k=2)
    episodic_memories = "\n".join([doc.page_content for doc in retrieved_docs])
    
    # Retrieve from semantic memory
    try:
        # Simple keyword-based retrieval from graph
        # Get all relationships for context
        semantic_result = graph.query("""
            MATCH (n)-[r]->(m)
            RETURN n.id as source, type(r) as relationship, m.id as target
            LIMIT 10
        """)
        if semantic_result:
            semantic_memories = "\n".join([f"{r['source']} --[{r['relationship']}]--> {r['target']}" for r in semantic_result])
        else:
            semantic_memories = "No semantic memories found."
    except Exception as e:
        semantic_memories = f"Could not query graph: {e}"
        
    retrieved_content = f"Relevant Past Conversations (Episodic Memory):\n{episodic_memories}\n\nRelevant Facts (Semantic Memory):\n{semantic_memories}"
    console.print(f"[cyan]Retrieved Context:\n{retrieved_content}[/cyan]")
    
    return {"retrieved_memories": retrieved_content}

def generate_response(state: AgentState) -> Dict[str, Any]:
    """Node that generates a response using the retrieved memories."""
    console.print("--- Generating Response ---")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful and personalized financial assistant. Use the retrieved memories to inform your response and tailor it to the user. If the memories indicate a user's preference (e.g., they are a conservative investor), you MUST respect it."),
        ("human", "My question is: {user_input}\n\nHere are some memories that might be relevant:\n{retrieved_memories}")
    ])
    generator = prompt | llm
    generation = generator.invoke(state).content
    console.print(f"[green]Generated Response:\n{generation}[/green]")
    return {"generation": generation}

def update_memory(state: AgentState) -> Dict[str, Any]:
    """Node that updates the memory with the latest interaction."""
    console.print("--- Updating Memory ---")
    create_memories(state['user_input'], state['generation'])
    return {}

# Build the graph
workflow = StateGraph(AgentState)

workflow.add_node("retrieve", retrieve_memory)
workflow.add_node("generate", generate_response)
workflow.add_node("update", update_memory)

workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", "update")
workflow.add_edge("update", END)

memory_agent = workflow.compile()
print("✅ Memory-augmented agent graph compiled successfully.")

✅ Memory-augmented agent graph compiled successfully.


## Phase 3: Demonstration & Inspection

Let's see the agent in action. We'll simulate a multi-turn conversation. The first two turns will seed the memory. The third turn will test if the agent can use that memory for a personalized response. Finally, we'll directly inspect the databases to see the memories that were created.

**Note:** Local models may be slower than cloud APIs. Be patient!

In [17]:
def run_interaction(query: str):
    result = memory_agent.invoke({"user_input": query})
    return result['generation']

console.print("\n--- 💬 INTERACTION 1: Seeding Memory ---")
run_interaction("Hi, my name is Alex. I'm a conservative investor, and I'm mainly interested in established tech companies.")

console.print("\n--- 💬 INTERACTION 2: Asking a specific question ---")
run_interaction("What do you think about Apple (AAPL)?")

console.print("\n--- 🧠 INTERACTION 3: THE MEMORY TEST ---")
run_interaction("Based on my goals, what's a good alternative to that stock?")

--- 💬 INTERACTION 1: Seeding Memory ---

--- Retrieving Memories ---

Retrieved Context:
Relevant Past Conversations (Episodic Memory):


Relevant Facts (Semantic Memory):
No semantic memories found.

--- Generating Response ---

Generated Response:
As a conservative investor focused on established tech companies, I’d emphasize stability, proven fundamentals, and
long-term growth. Here’s how to approach them:

### **Key Criteria for Established Tech Companies**  
1. **Strong Financials**: Look for companies with consistent revenue growth, healthy profit margins, and low debt. 
2. **Track Record**: Prioritize firms with decades of operation (e.g., Microsoft, Apple, Amazon) or industry 
leaders (e.g., Intel, IBM) with a history of reliable performance.  
3. **Dividend Payments**: Consider companies offering regular dividends (e.g., Coca-Cola, Procter & Gamble) to 
align with conservative preferences.  
4. **Low Valuation**: Focus on undervalued stocks with strong earnings potential (e.g., NVIDIA, AMD) or companies 
in mature sectors (e.g., software, hardware).  

### **Suggested Sectors**  
- **Software**: Microsoft (Azure, Office 365), Salesforce (cloud solutions).  
- **Hardware**: Intel (processors), AMD (chipset), Apple (macOS, iOS).  
- **Services**: Amazon (cloud services), Google (Alphabet, cloud infrastructure).  

### **Tips for Research**  
- **Industry Trends**: Watch for shifts in AI, cloud computing, or sustainability (e.g., Tesla, Siemens).  
- **Competitive Advantage**: Look for companies with patents, proprietary technology, or dominant market positions.
- **Valuation Metrics**: Use metrics like P/E ratios, earnings per share (EPS), and return on equity (ROE) to 
assess relative value.  

Would you like me to refine your list of candidates or highlight specific sectors? Always remember to balance risk 
with reward and consult a financial advisor for personalized guidance.

...

--- Creating Episodic Memory (Summary) ---

Episodic memory created: 'The assistant recommends Intel, AMD, and Microsoft as conservative alternatives to Apple 
(AAPL) with varying risk profiles and sectors, emphasizing diversification, valuation monitoring, and 
sector-specific considerations.'

--- Creating Semantic Memory (Graph) ---

Semantic memory created: Added 5 relationships to the graph.

'Based on your conservative investment goals and interest in established tech companies, here are some strong alternatives to Apple (AAPL) that align with your preferences:\n\n### **1. Intel (INTC)**  \n- **Why it fits**: A long-standing, high-quality semiconductor company with a robust ecosystem, strong financials, and a competitive edge in CPU and chip manufacturing.  \n- **Risk profile**: Moderate volatility due to semiconductor demand and geopolitical factors, but historically stable in value.  \n- **Sector**: Hardware (semiconductors).  \n\n### **2. AMD (AMD)**  \n- **Why it fits**: A leader in GPU and CPU technology, with a strong track record of innovation and a diversified revenue model (e.g., gaming, data centers, and AI).  \n- **Risk profile**: Slightly more volatile than Intel due to gaming and AI demand cycles, but still offers solid fundamentals.  \n- **Sector**: Hardware (GPUs, CPUs).  \n\n### **3. Microsoft (MSFT)**  \n- **Why it fits**: A software giant with a vast ecos

### Inspecting the Memory Stores

Let's look under the hood. We can query our databases directly to see the memories our agent created.

In [18]:
console.print("--- 🔍 Inspecting Episodic Memory (Vector Store) ---")
# We'll do a similarity search for a general concept to see what comes up
retrieved_docs = episodic_vector_store.similarity_search("User's investment strategy", k=5)
for i, doc in enumerate(retrieved_docs):
    print(f"{i+1}. {doc.page_content}")

console.print("\n--- 🕸️ Inspecting Semantic Memory (Graph Database) ---")
print(f"Graph Schema:\n{graph.get_schema}")

# Cypher query to see all relationships
query_result = graph.query("MATCH (n)-[r]->(m) RETURN n.id as source, labels(n) as source_type, type(r) as relationship, m.id as target, labels(m) as target_type")
print(f"\nRelationships in Graph:")
for r in query_result:
    print(f"  ({r['source']}:{r['source_type']}) --[{r['relationship']}]--> ({r['target']}:{r['target_type']})")

--- 🔍 Inspecting Episodic Memory (Vector Store) ---

1. Alex, a conservative investor in established tech companies, seeks stability and proven fundamentals, prioritizing sectors like software, hardware, and services with strong financials, low valuations, and competitive advantages, while balancing risk and reward through industry trends and valuation metrics.
2. Apple (AAPL) is a high-quality, low-risk investment for conservative investors, offering strong financials, a robust ecosystem, and stable growth, but with valuation concerns and market volatility that require careful diversification and monitoring.
3. The assistant recommends Intel, AMD, and Microsoft as conservative alternatives to Apple (AAPL) with varying risk profiles and sectors, emphasizing diversification, valuation monitoring, and sector-specific considerations.


--- 🕸️ Inspecting Semantic Memory (Graph Database) ---

Graph Schema:
Node properties:
User {id: STRING}
InvestmentPhilosophy {id: STRING}
Company {id: STRING}
Assistant {id: STRING}
Topic {id: STRING}
Sector {id: STRING}
Relationship properties:

The relationships:
(:User)-[:PREFERS]->(:Sector)
(:User)-[:PREFERS]->(:InvestmentPhilosophy)
(:User)-[:INTERESTED_IN]->(:Company)
(:User)-[:INTERESTED_IN]->(:InvestmentPhilosophy)
(:User)-[:HAS_GOAL]->(:Company)
(:InvestmentPhilosophy)-[:PREFERS]->(:InvestmentPhilosophy)
(:InvestmentPhilosophy)-[:HAS_GOAL]->(:InvestmentPhilosophy)
(:InvestmentPhilosophy)-[:MENTIONED]->(:InvestmentPhilosophy)
(:Company)-[:MENTIONED]->(:Topic)
(:Assistant)-[:MENTIONED]->(:Topic)
(:Assistant)-[:PREFERS]->(:Sector)

Relationships in Graph:
  (User:['User']) --[INTERESTED_IN]--> (Company:['Company'])
  (User:['User']) --[HAS_GOAL]--> (Company:['Company'])
  (User:['User']) --[HAS_GOAL]--> (Assistant:['Assistant'])
  (User:['User']) --[INTERESTED_IN]--> (Apple (AAPL):['Company'])
  (User:['User']) --[INTERESTED_IN]--> (

### Try Your Own Query

Run the cell below to interact with the agent yourself!

In [19]:
# Try your own query!
your_query = "What do you remember about me?"

console.print(f"\n--- 💬 Your Query: {your_query} ---")
response = run_interaction(your_query)
print(f"\n📝 Final Response:\n{response}")

--- 💬 Your Query: What do you remember about me? ---

--- Retrieving Memories ---

Retrieved Context:
Relevant Past Conversations (Episodic Memory):
Apple (AAPL) is a high-quality, low-risk investment for conservative investors, offering strong financials, a 
robust ecosystem, and stable growth, but with valuation concerns and market volatility that require careful 
diversification and monitoring.
Alex, a conservative investor in established tech companies, seeks stability and proven fundamentals, prioritizing 
sectors like software, hardware, and services with strong financials, low valuations, and competitive advantages, 
while balancing risk and reward through industry trends and valuation metrics.

Relevant Facts (Semantic Memory):
User --[INTERESTED_IN]--> Company
User --[HAS_GOAL]--> Company
User --[HAS_GOAL]--> Assistant
User --[INTERESTED_IN]--> Apple (AAPL)
User --[INTERESTED_IN]--> Assistant
User --[MENTIONED]--> Sectors
User --[PREFERS]--> Preference
User --[HAS_GOAL]--> Conservative Investment Philosophy
User --[INTERESTED_IN]--> established tech companies
Assistant --[MENTIONED]--> NVIDIA

--- Generating Response ---

Generated Response:
I remember that you're a **conservative investor** with a focus on **established tech companies** and 
**high-quality, low-risk investments** like Apple (AAPL). You prioritize sectors with **strong financials, low 
valuations, and competitive advantages**—such as software, hardware, and services. You also value **industry 
trends** and **valuation metrics** to balance risk and reward.  

You’ve shown interest in **Apple (AAPL)** and **NVIDIA** (as mentioned in previous conversations), which aligns 
with your conservative philosophy. You likely seek stability, proven fundamentals, and long-term growth while 
avoiding overvaluation or excessive volatility.  

Would you like me to tailor advice based on your specific goals, risk tolerance, or portfolio? 😊

...

--- Creating Episodic Memory (Summary) ---

Episodic memory created: 'The user is a conservative investor focused on established tech companies like Apple 
(AAPL) and NVIDIA, prioritizing strong financials, low risk, industry trends, and valuation metrics, seeking 
stability, proven fundamentals, and long-term growth, with the assistant ready to tailor advice based on specific 
goals and risk tolerance.'

--- Creating Semantic Memory (Graph) ---

Semantic memory created: Added 7 relationships to the graph.


📝 Final Response:
I remember that you're a **conservative investor** with a focus on **established tech companies** and **high-quality, low-risk investments** like Apple (AAPL). You prioritize sectors with **strong financials, low valuations, and competitive advantages**—such as software, hardware, and services. You also value **industry trends** and **valuation metrics** to balance risk and reward.  

You’ve shown interest in **Apple (AAPL)** and **NVIDIA** (as mentioned in previous conversations), which aligns with your conservative philosophy. You likely seek stability, proven fundamentals, and long-term growth while avoiding overvaluation or excessive volatility.  

Would you like me to tailor advice based on your specific goals, risk tolerance, or portfolio? 😊
